[medium](https://medium.com/@jonathan_hui/rl-dqn-deep-q-network-e207751f7ae4) <br>
[ref](https://github.com/udacity/deep-reinforcement-learning/tree/master/dqn)

In [1]:
%reload_ext autoreload
%autoreload 2
import torch
import torch.nn.functional as F 
import random
import numpy as np
from EXITrl.approx_v_base import ApproxVBase
from EXITrl.approx_policy_base import ApproxPolicyBase
from EXITrl.base import Base
from EXITrl.helpers import print_weight_size, copy_params, update_params, ExperienceReplay, convert_to_tensor
from EXITrl.nn_wrapper import NNWrapper
import gym

In [2]:
class QNetwork(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        self.linear1 = torch.nn.Linear(input_size, hidden_size)
        self.linear2 = torch.nn.Linear(hidden_size, hidden_size)
        self.linear3 = torch.nn.Linear(hidden_size, output_size)

    def forward(self, state):
        x = F.relu(self.linear1(state))
        x = F.relu(self.linear2(x))
        return self.linear3(x)

In [28]:
class DQN(Base):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.local_q_network = NNWrapper(
            QNetwork(self.num_state, 64, self.num_action),
            lr=self.alpha
        )
        self.target_q_network = NNWrapper(
            QNetwork(self.num_state, 64, self.num_action),
            lr=self.beta
        )
        self.experience_replay = ExperienceReplay(num_experience=2048)
        
        self.update_count = 0
        self.epsilon = 1

    def _loop(self, episode) -> int:
        done = False
        total_reward, reward = 0, 0
        state = self.env.reset()
        for i in range(1000):
            self.epsilon *= 0.99
            if self.epsilon<0.01:
                self.epsilon= 0.01
            
            action = self.local_q_network.epsilon_greedy(state, self.epsilon)
            _state, reward, done, _ = self.env.step(action)
            self.experience_replay.remember(state, action, reward, _state, done)
            
            if self.update_count%4 == 0:
                batch_state, \
                batch_action, \
                batch_reward, \
                batch_next_state, \
                batch_done = self.experience_replay.recall(batch_size=64)

                # detach because we only backprop local network and update target network weight manually
                targets_next_Q = self.target_q_network.forward(batch_next_state).detach().max(1)[0]
                targets_Q = batch_reward + (self.gamma * targets_next_Q * (1 - batch_done))

                local_Q = self.local_q_network.forward(batch_state)
                expected_Q = local_Q.gather(1, batch_action.unsqueeze(1).long()).squeeze(1)

                loss = F.mse_loss(expected_Q, targets_Q)
                self.local_q_network.backprop(loss)
            
                update_params(self.local_q_network.model, self.target_q_network.model, self.tau)
            
            total_reward += reward
            state = _state
            self.update_count += 1
            if done: return total_reward
        
try: env.close()
except: pass
env = gym.make('LunarLander-v2')
dqn = DQN(env, 
      num_episodes=1000,
      alpha=0.0001, 
      beta=0.0001,
      gamma=.99)
dqn.train(True)

episode: 0 reward: -226.44267540066366
episode: 1 reward: -326.62694074050955
episode: 2 reward: -351.1618916365005
episode: 3 reward: -238.70116875024559
episode: 4 reward: -358.52970085684
episode: 5 reward: -705.1513261532389
episode: 6 reward: -902.6289750560985
episode: 7 reward: -542.8400062226499
episode: 8 reward: -325.94967965055076
episode: 9 reward: -195.68620219895047
episode: 10 reward: -157.03485895256097
episode: 11 reward: -155.07345102613698
episode: 12 reward: -335.19486260536985
episode: 13 reward: -107.28955070040168
episode: 14 reward: -134.5242711741377
episode: 15 reward: -115.10715352842595
episode: 16 reward: -120.41377269354793
episode: 17 reward: -117.69433396457848
episode: 18 reward: -121.25979196350139
episode: 19 reward: -121.03164120309083
episode: 20 reward: -127.62192166571037
episode: 21 reward: -124.48268513541106
episode: 22 reward: -133.4827280362311
episode: 23 reward: -150.30187188751324
episode: 24 reward: -218.83925985542567
episode: 25 reward:

episode: 206 reward: -137.77467800709184
episode: 207 reward: -125.60115862217208
episode: 208 reward: -129.32349369158774
episode: 209 reward: -178.00068452048774
episode: 210 reward: -145.66206134201934
episode: 211 reward: -164.87384316988278
episode: 212 reward: -170.50260366363625
episode: 213 reward: -199.84084315623477
episode: 214 reward: -159.50350787057462
episode: 215 reward: -232.02855736860747
episode: 216 reward: -278.99987668252874
episode: 217 reward: -98.60868014719193
episode: 218 reward: -365.3060770387167
episode: 219 reward: -170.91583462184525
episode: 220 reward: -148.82016705236748
episode: 221 reward: -168.37681030627596
episode: 222 reward: -175.1514081332245
episode: 223 reward: -157.879783313023
episode: 224 reward: -164.79724712656156
episode: 225 reward: -300.6455830229578
episode: 226 reward: -126.35905508005511
episode: 227 reward: -162.70301297985958
episode: 228 reward: -181.11504333439706
episode: 229 reward: -149.1370178663248
episode: 230 reward: -1

episode: 409 reward: -148.30256480716793
episode: 410 reward: -119.31309187060631
episode: 411 reward: -99.78940674001292
episode: 412 reward: -132.1106521327839
episode: 413 reward: -146.477423133122
episode: 414 reward: -127.55454097288319
episode: 415 reward: -114.00437696207915
episode: 416 reward: -107.94647165490437
episode: 417 reward: -128.33730220768373
episode: 418 reward: -112.93548788396288
episode: 419 reward: 42.76650762490968
episode: 420 reward: -137.92407585215415
episode: 421 reward: -116.5660139563826
episode: 422 reward: -137.03539926835387
episode: 423 reward: -143.95185434732588
episode: 424 reward: -111.46743941670094
episode: 425 reward: -128.33397090945903
episode: 426 reward: -118.68188008180712
episode: 427 reward: -113.37358116261329
episode: 428 reward: -123.70026015352221
episode: 429 reward: -116.94949689615444
episode: 430 reward: -111.88994466994444
episode: 431 reward: -112.05394892223885
episode: 432 reward: -134.79157676482887
episode: 433 reward: -1

episode: 611 reward: -113.0613085745108
episode: 612 reward: -123.22337174122457
episode: 613 reward: -108.67452497163957
episode: 614 reward: -119.71435542135882
episode: 615 reward: -107.14335401180472
episode: 616 reward: -129.20344846546325
episode: 617 reward: -110.10206539743844
episode: 618 reward: -122.5788954022165
episode: 619 reward: -88.11924520589321
episode: 620 reward: -120.92448315662824
episode: 621 reward: -138.2802241229189
episode: 622 reward: -99.64729789735574
episode: 623 reward: -97.91218362565016
episode: 624 reward: -115.89548128831576
episode: 625 reward: -119.38063583394128
episode: 626 reward: -107.96253319688587
episode: 627 reward: -135.0787032307579
episode: 628 reward: -130.33369881938503
episode: 629 reward: -85.09681796091931
episode: 630 reward: -109.60725626735979
episode: 631 reward: -125.80957618225685
episode: 632 reward: -106.03858697979068
episode: 633 reward: -107.45873084621162
episode: 634 reward: -86.92326791612825
episode: 635 reward: -123

episode: 813 reward: -103.35232151771717
episode: 814 reward: -190.45651245178726
episode: 815 reward: -345.6810050257145
episode: 816 reward: -311.912687707654
episode: 817 reward: -48.33745366682152
episode: 818 reward: -109.64648540477249
episode: 819 reward: -221.1691327905677
episode: 820 reward: -123.99609128879615
episode: 821 reward: -112.01311097186367
episode: 822 reward: -102.15701000689243
episode: 823 reward: -105.57437169437841
episode: 824 reward: -80.43610244534717
episode: 825 reward: -89.5470332372083
episode: 826 reward: -90.219439694396
episode: 827 reward: -104.20096492324282
episode: 828 reward: -124.51476009650096
episode: 829 reward: -95.96700907893475
episode: 830 reward: -85.6133824521793
episode: 831 reward: -112.32393128726875
episode: 832 reward: -94.65260259580646
episode: 833 reward: -115.05370575452734
episode: 834 reward: -95.05513341765447
episode: 835 reward: -84.492504518006
episode: 836 reward: -222.5723474359507
episode: 837 reward: -95.84150245596

In [11]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.tensor([1,2], dtype=torch.float, device=device)

tensor([1., 2.], device='cuda:0')

In [29]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class QNetwork(nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, seed, fc1_units=64, fc2_units=64):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            seed (int): Random seed
            fc1_units (int): Number of nodes in first hidden layer
            fc2_units (int): Number of nodes in second hidden layer
        """
        super(QNetwork, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, fc2_units)
        self.fc3 = nn.Linear(fc2_units, action_size)

    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        x = F.relu(self.fc2(x))
        return self.fc3(x)


In [30]:
import numpy as np
import random
from collections import namedtuple, deque

import torch
import torch.nn.functional as F
import torch.optim as optim

BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size, seed).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size, seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Tensor]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences

        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))

        # Get expected Q values from local model
        Q_expected = self.qnetwork_local(states).gather(1, actions)

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)


class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)


In [31]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline
env = gym.make('LunarLander-v2')
env.seed(0)
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space.n)

def dqn(n_episodes=2000, max_t=1000, eps_start=1.0, eps_end=0.01, eps_decay=0.995):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    eps = eps_start                    # initialize epsilon
    for i_episode in range(1, n_episodes+1):
        state = env.reset()
        score = 0
        for t in range(max_t):
            action = agent.act(state, eps)
            next_state, reward, done, _ = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state
            score += reward
            if done:
                break 
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)), end="")
        if i_episode % 100 == 0:
            print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
        if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
            break
    return scores

agent = Agent(state_size=8, action_size=4, seed=0)
scores = dqn()

# plot the scores
fig = plt.figure()
ax = fig.add_subplot(111)
plt.plot(np.arange(len(scores)), scores)
plt.ylabel('Score')
plt.xlabel('Episode #')
plt.show()

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


State shape:  (8,)
Number of actions:  4
Episode 100	Average Score: -203.67
Episode 200	Average Score: -87.983
Episode 299	Average Score: -77.68

KeyboardInterrupt: 